<a target="_blank" href="https://colab.research.google.com/github/victorlymarev/pandas/blob/main/notebooks/17-groupby.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path_empl = '../tables/employees.parquet' if os.path.exists('../tables/employees.parquet') else 'https://drive.google.com/uc?id=1AARD5-eVlCxoApt5CYZebrC3Cqw42lvj'

empl = pd.read_parquet(path_empl).drop(['pos_id', 'closest_boss_pos_id', 'mgmt_flag', 'salary_fork'], axis=1)
empl.head()

,report_dt,i_pernr,fio,sex,shop_id,pos_name,birth_date,education,employee_evaluation,salary
0,2015-01-31,100000,Кожевников Андрей Максимович,мужской,1,Директор магазина,1987-03-11,высшее,5,45700.0
1,2015-01-31,0,Дубинина Надежда Николаевна,женский,1,Старший кассир,1971-07-07,среднее профессиональное,5,20000.0
2,2015-01-31,1,Вишневскиая Валентина Константиновна,женский,1,Кассир,1974-11-29,среднее общее,5,12600.0
3,2015-01-31,2,Литвинов Александр Максимович,мужской,1,Кассир,1969-11-01,среднее общее,3,12600.0
4,2015-01-31,3,Черняева Валерия Глебовна,женский,1,Продавец-консультант,1988-07-17,среднее общее,<NA>,15400.0


### Задача: посчитать среднюю зарплату за каждый месяц

In [3]:
empl['report_dt'].unique()[:10]

array(['2015-01-31T00:00:00.000000000', '2015-02-28T00:00:00.000000000',
       '2015-03-31T00:00:00.000000000', '2015-04-30T00:00:00.000000000',
       '2015-05-31T00:00:00.000000000', '2015-06-30T00:00:00.000000000',
       '2015-07-31T00:00:00.000000000', '2015-08-31T00:00:00.000000000',
       '2015-09-30T00:00:00.000000000', '2015-10-31T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [4]:
empl.query("report_dt == '2015-01-31'")['salary'].mean()

18330.0

In [5]:
# Создаем пустой список в который будем класть зарплату за каждый месяц
avg_salary = []

# идем циклом по каждой дате
for date in empl['report_dt'].unique():
#     добавляем среднюю зп для каждой даты
    avg_salary.append(empl.query("report_dt == @date")['salary'].mean())

# Кладем ее в датафрейм
pd.DataFrame({'Дата': empl['report_dt'].unique(),
              'Средняя зарплата': avg_salary}).head()

,Дата,Средняя зарплата
0,2015-01-31,18330.000000
1,2015-02-28,17625.000000
2,2015-03-31,17253.846154
3,2015-04-30,27735.483871
4,2015-05-31,27294.594595


In [6]:
(empl
    .groupby('report_dt') # Группируем по дате
    ['salary'] # вибираем колонку
    .mean() # считаем по ней среднее
    .head()
)

report_dt
2015-01-31    18330.000000
2015-02-28    17625.000000
2015-03-31    17253.846154
2015-04-30    27735.483871
2015-05-31    27294.594595
Name: salary, dtype: float64

In [7]:
(empl
    .groupby('report_dt')
    [['salary']] # название колонки лежит в списке
    .mean()
    .head()
)

,salary
report_dt,
2015-01-31,18330.000000
2015-02-28,17625.000000
2015-03-31,17253.846154
2015-04-30,27735.483871
2015-05-31,27294.594595


#### Убираем колонку по которой шла группировка из индекса

In [8]:
(empl
    .groupby('report_dt')
    ['salary']
    .mean()
    .reset_index() # убираем столбец report_dt из индекса
    .head()
)

,report_dt,salary
0,2015-01-31,18330.000000
1,2015-02-28,17625.000000
2,2015-03-31,17253.846154
3,2015-04-30,27735.483871
4,2015-05-31,27294.594595


#### Параметр as_index

In [10]:
(empl
    .groupby('report_dt', as_index=False) # То же самое, что и reset_index после метода mean()
    ['salary']
    .mean()
    .head()
)

,report_dt,salary
0,2015-01-31,18330.000000
1,2015-02-28,17625.000000
2,2015-03-31,17253.846154
3,2015-04-30,27735.483871
4,2015-05-31,27294.594595


#### Параметр dropna

In [11]:
(empl
    .groupby('employee_evaluation', dropna=False)
    ['salary']
    .mean()
)

employee_evaluation
1       35800.000000
2       30678.260870
3       31094.670846
4       31709.630412
5       32895.414921
6       32199.660710
7       36856.810387
8       43006.856187
9       47333.223684
10      38716.000000
<NA>    28558.353909
Name: salary, dtype: float64

#### Несколько агрегаций

In [12]:
(empl
    .groupby('report_dt')
    ['salary']
    .agg(['mean', 'std', lambda x: (x - x.median()).abs().mean()]) # список из функций агрегаций
    .head()
)

,mean,std,<lambda_0>
report_dt,,,
2015-01-31,18330.000000,9826.726594,4050.000000
2015-02-28,17625.000000,9059.512831,3608.333333
2015-03-31,17253.846154,8776.447211,3530.769231
2015-04-30,27735.483871,18280.473159,10667.741935
2015-05-31,27294.594595,16738.610488,9140.540541


#### Агрегация по нескольким колонкам

In [13]:
(empl
    .groupby('report_dt')
    [['salary', 'birth_date']]
    .agg(['min', 'max'])
    .head()
)

salary          birth_date           
                min      max        min        max
report_dt                                         
2015-01-31  12600.0  45700.0 1966-02-18 1988-07-17
2015-02-28  12600.0  45700.0 1966-02-18 1990-11-16
2015-03-31  12600.0  45700.0 1965-12-01 1990-11-16
2015-04-30  12600.0  94000.0 1962-10-15 1996-11-04
2015-05-31  12600.0  94000.0 1962-10-15 1996-11-04

In [14]:
(empl
    .groupby('report_dt')

    .agg({'salary': ['min', 'max'],
          'birth_date': 'median'}) 
    .head()
)

salary                   birth_date
                min      max              median
report_dt                                       
2015-01-31  12600.0  45700.0 1979-03-25 12:00:00
2015-02-28  12600.0  45700.0 1984-07-24 00:00:00
2015-03-31  12600.0  45700.0 1983-07-20 00:00:00
2015-04-30  12600.0  94000.0 1983-07-20 00:00:00
2015-05-31  12600.0  94000.0 1983-03-07 00:00:00

### Группировка по нескольким колонкам

In [15]:
(empl
    .groupby(['report_dt', 'pos_name']) # по дате и по должности
    ['salary']
    .mean()
    .head()
)

report_dt   pos_name            
2015-01-31  Директор магазина       45700.0
            Кассир                  12600.0
            Продавец-консультант    15400.0
            Старший кассир          20000.0
2015-02-28  Директор магазина       45700.0
Name: salary, dtype: float64

In [16]:
(empl
    .groupby(['report_dt', 'pos_name']) # по дате и по должности
    [['salary', 'employee_evaluation']]
    .mean()
    .head()
)

salary  employee_evaluation
report_dt  pos_name                                          
2015-01-31 Директор магазина     45700.0                  5.0
           Кассир                12600.0                  4.0
           Продавец-консультант  15400.0                  4.5
           Старший кассир        20000.0                  5.0
2015-02-28 Директор магазина     45700.0                  5.0

#### Добавление дополнителных колонок, которые не влияют на саму группировку

In [17]:
(empl
    .groupby('i_pernr')
    [['salary']]
    .max()
    .reset_index()
    .head()
)

,i_pernr,salary
0,0,32100.0
1,1,21400.0
2,2,15200.0
3,3,28600.0
4,4,28600.0


In [18]:
(empl
    .groupby(['i_pernr', 'fio'])
    [['salary']]
    .max()
    .reset_index()
    .head()
)

,i_pernr,fio,salary
0,0,Дубинина Надежда Николаевна,32100.0
1,1,Вишневскиая Валентина Константиновна,21400.0
2,2,Литвинов Александр Максимович,15200.0
3,3,Черняева Валерия Глебовна,28600.0
4,4,Никифоров Даниил Глебович,28600.0


In [19]:
(empl
    .groupby(['i_pernr', 'fio', 'birth_date'])
    [['salary']]
    .max()
    .reset_index()
    .head()
)

,i_pernr,fio,birth_date,salary
0,0,Дубинина Надежда Николаевна,1971-07-07,32100.0
1,1,Вишневскиая Валентина Константиновна,1974-11-29,21400.0
2,2,Литвинов Александр Максимович,1969-11-01,15200.0
3,3,Черняева Валерия Глебовна,1988-07-17,28600.0
4,4,Никифоров Даниил Глебович,1967-10-01,28600.0


#### Приведение таблицы к виду сводной таблицы

In [21]:
(empl
    .groupby(['report_dt', 'pos_name'])
    ['salary']
    .mean()
    .unstack('pos_name') # перебрасываем pos_name в шапку таблицы
    .head()
)

pos_name,Бухгалтер,Директор магазина,Заместитель директора магазина,Кассир,Мастер по ремонту одежды,Продавец-консультант,Старший кассир,Товаровед,Уборщик
report_dt,,,,,,,,,
2015-01-31,NaN,45700.0,NaN,12600.000000,NaN,15400.000000,20000.0,NaN,NaN
2015-02-28,NaN,45700.0,NaN,12633.333333,NaN,15414.285714,20000.0,NaN,NaN
2015-03-31,NaN,45700.0,NaN,12675.000000,NaN,15414.285714,20000.0,NaN,NaN
2015-04-30,32300.0,69850.0,67600.0,18987.500000,32300.0,20746.153846,27050.0,22300.0,NaN
2015-05-31,32300.0,69850.0,67600.0,20736.363636,32300.0,21526.666667,27050.0,22300.0,20700.0


Как и другие методы, работающие с уровнем индекса, метод unstack принимает на вход номер уровня

In [22]:
(empl
    .groupby(['report_dt', 'pos_name'])
    ['salary']
    .mean()
    .unstack(1)
    .head()
)

pos_name,Бухгалтер,Директор магазина,Заместитель директора магазина,Кассир,Мастер по ремонту одежды,Продавец-консультант,Старший кассир,Товаровед,Уборщик
report_dt,,,,,,,,,
2015-01-31,NaN,45700.0,NaN,12600.000000,NaN,15400.000000,20000.0,NaN,NaN
2015-02-28,NaN,45700.0,NaN,12633.333333,NaN,15414.285714,20000.0,NaN,NaN
2015-03-31,NaN,45700.0,NaN,12675.000000,NaN,15414.285714,20000.0,NaN,NaN
2015-04-30,32300.0,69850.0,67600.0,18987.500000,32300.0,20746.153846,27050.0,22300.0,NaN
2015-05-31,32300.0,69850.0,67600.0,20736.363636,32300.0,21526.666667,27050.0,22300.0,20700.0


### crosstab
Функция для формирования сводной таблицы

In [23]:
pd.crosstab(index=empl['report_dt'], # колонка в индексе
            columns=empl['pos_name'], # колонка, которая будет в шапке таблицы
            values=empl['salary'], # колонка, считается среднее
            aggfunc='mean' # функция агрегации
           ).head()

pos_name,Бухгалтер,Директор магазина,Заместитель директора магазина,Кассир,Мастер по ремонту одежды,Продавец-консультант,Старший кассир,Товаровед,Уборщик
report_dt,,,,,,,,,
2015-01-31,NaN,45700.0,NaN,12600.000000,NaN,15400.000000,20000.0,NaN,NaN
2015-02-28,NaN,45700.0,NaN,12633.333333,NaN,15414.285714,20000.0,NaN,NaN
2015-03-31,NaN,45700.0,NaN,12675.000000,NaN,15414.285714,20000.0,NaN,NaN
2015-04-30,32300.0,69850.0,67600.0,18987.500000,32300.0,20746.153846,27050.0,22300.0,NaN
2015-05-31,32300.0,69850.0,67600.0,20736.363636,32300.0,21526.666667,27050.0,22300.0,20700.0


### pivot_table 
Метод для формирования сводной таблицы

In [24]:
empl.pivot_table(index='report_dt', # колонка в индексе
            columns='pos_name', # колонка, которая будет в шапке таблицы
            values='salary', # колонка, считается среднее
            aggfunc='mean' # функция агрегации
                ).head()

pos_name,Бухгалтер,Директор магазина,Заместитель директора магазина,Кассир,Мастер по ремонту одежды,Продавец-консультант,Старший кассир,Товаровед,Уборщик
report_dt,,,,,,,,,
2015-01-31,NaN,45700.0,NaN,12600.000000,NaN,15400.000000,20000.0,NaN,NaN
2015-02-28,NaN,45700.0,NaN,12633.333333,NaN,15414.285714,20000.0,NaN,NaN
2015-03-31,NaN,45700.0,NaN,12675.000000,NaN,15414.285714,20000.0,NaN,NaN
2015-04-30,32300.0,69850.0,67600.0,18987.500000,32300.0,20746.153846,27050.0,22300.0,NaN
2015-05-31,32300.0,69850.0,67600.0,20736.363636,32300.0,21526.666667,27050.0,22300.0,20700.0


In [25]:
empl.pivot_table(index='report_dt',
            columns='pos_name',
            values='salary',
            aggfunc='mean',
            margins=True # Можно добавить аргрегицию по подгруппе
                ).iloc[[10, 20, -2, -1], [0, 1, 2, -2, -1]]

pos_name,Бухгалтер,Директор магазина,Заместитель директора магазина,Уборщик,All
report_dt,,,,,
2015-11-30 00:00:00,32300.000000,64333.333333,67600.000000,20700.000000,29358.333333
2016-09-30 00:00:00,37150.000000,82920.000000,65840.000000,22250.000000,28562.831858
2022-12-31 00:00:00,48900.000000,114960.869565,80523.809524,27120.000000,37754.414784
All,42045.859873,99779.370629,71885.007849,22973.870968,33200.401743


#### Агрегация нескольких колонок в одной сводной таблице

In [26]:
(empl
    .groupby(['report_dt', 'pos_name'])
    [['salary', 'employee_evaluation']] # зп и оценка сотрудника 
    .mean()
    .unstack(1)
    .head()
)

salary                                 \
pos_name   Бухгалтер                      Директор магазина                
report_dt                                                                  
2015-01-31                            NaN                        45700.0   
2015-02-28                            NaN                        45700.0   
2015-03-31                            NaN                        45700.0   
2015-04-30                        32300.0                        69850.0   
2015-05-31                        32300.0                        69850.0   

                                                                          \
pos_name   Заместитель директора магазина Кассир                           
report_dt                                                                  
2015-01-31                            NaN                   12600.000000   
2015-02-28                            NaN                   12633.333333   
2015-03-31                            NaN                   12675.000000   
2015-04-30                        67600.0                   18987.500000   
2015-05-31                        67600.0                   20736.363636   

                                                                          \
pos_name   Мастер по ремонту одежды       Продавец-консультант             
report_dt                                                                  
2015-01-31                            NaN                   15400.000000   
2015-02-28                            NaN                   15414.285714   
2015-03-31                            NaN                   15414.285714   
2015-04-30                        32300.0                   20746.153846   
2015-05-31                        32300.0                   21526.666667   

                                                                          \
pos_name   Старший кассир                 Товаровед                        
report_dt                                                                  
2015-01-31                        20000.0                            NaN   
2015-02-28                        20000.0                            NaN   
2015-03-31                        20000.0                            NaN   
2015-04-30                        27050.0                        22300.0   
2015-05-31                        27050.0                        22300.0   

                                                     employee_evaluation  \
pos_name   Уборщик                        Бухгалтер                        
report_dt                                                                  
2015-01-31                            NaN                           <NA>   
2015-02-28                            NaN                           <NA>   
2015-03-31                            NaN                           <NA>   
2015-04-30                            NaN                           <NA>   
2015-05-31                        20700.0                            4.0   

                                                                          \
pos_name   Директор магазина              Заместитель директора магазина   
report_dt                                                                  
2015-01-31                            5.0                           <NA>   
2015-02-28                            5.0                           <NA>   
2015-03-31                            5.0                           <NA>   
2015-04-30                            5.5                           <NA>   
2015-05-31                            5.0                            6.0   

                                                                          \
pos_name   Кассир                         Мастер по ремонту одежды         
report_dt                                                                  
2015-01-31                            4.0                           <NA>   
2015-02-28                            4.5                           <NA>   
2

То же самое методом pivot_table

In [27]:
empl.pivot_table(index='report_dt',
            columns='pos_name',
            values=['salary', 'employee_evaluation'], # список из колонок
            aggfunc='mean').head()

employee_evaluation                                 \
pos_name   Бухгалтер                      Директор магазина                
report_dt                                                                  
2015-01-31                           <NA>                            5.0   
2015-02-28                           <NA>                            5.0   
2015-03-31                           <NA>                            5.0   
2015-04-30                           <NA>                            5.5   
2015-05-31                            4.0                            5.0   

                                                                          \
pos_name   Заместитель директора магазина Кассир                           
report_dt                                                                  
2015-01-31                           <NA>                            4.0   
2015-02-28                           <NA>                            4.5   
2015-03-31                           <NA>                           5.25   
2015-04-30                           <NA>                            5.0   
2015-05-31                            6.0                       5.222222   

                                                                          \
pos_name   Мастер по ремонту одежды       Продавец-консультант             
report_dt                                                                  
2015-01-31                           <NA>                            4.5   
2015-02-28                           <NA>                       5.571429   
2015-03-31                           <NA>                       5.714286   
2015-04-30                           <NA>                       5.714286   
2015-05-31                            6.0                            5.0   

                                                                          \
pos_name   Старший кассир                 Товаровед                        
report_dt                                                                  
2015-01-31                            5.0                           <NA>   
2015-02-28                            5.0                           <NA>   
2015-03-31                            5.0                           <NA>   
2015-04-30                            4.5                            4.0   
2015-05-31                            5.0                            4.5   

                                                                  salary  \
pos_name   Уборщик                        Бухгалтер                        
report_dt                                                                  
2015-01-31                           <NA>                            NaN   
2015-02-28                           <NA>                            NaN   
2015-03-31                           <NA>                            NaN   
2015-04-30                           <NA>                        32300.0   
2015-05-31                            5.0                        32300.0   

                                                                          \
pos_name   Директор магазина              Заместитель директора магазина   
report_dt                                                                  
2015-01-31                        45700.0                            NaN   
2015-02-28                        45700.0                            NaN   
2015-03-31                        45700.0                            NaN   
2015-04-30                        69850.0                        67600.0   
2015-05-31                        69850.0                        67600.0   

                                                                          \
pos_name   Кассир                         Мастер по ремонту одежды         
report_dt                                                                  
2015-01-31                   12600.000000                            NaN   
2015-02-28                   12633.333333                       

## groupby(...).transform

Трансформирует данные

In [28]:
(empl
    .groupby(['report_dt', 'pos_name'])
    ['salary']
    .agg('mean')
    .head()
)

report_dt   pos_name            
2015-01-31  Директор магазина       45700.0
            Кассир                  12600.0
            Продавец-консультант    15400.0
            Старший кассир          20000.0
2015-02-28  Директор магазина       45700.0
Name: salary, dtype: float64

In [29]:
(empl
    .groupby(['report_dt', 'pos_name'])
    ['salary']
    .transform('mean')
    .head()
)

0    45700.0
1    20000.0
2    12600.0
3    12600.0
4    15400.0
Name: salary, dtype: float64

Добавим колонку, в которой будет содержаться средняя зарплата внутри группы

In [30]:
(empl
    .assign(mean_group_salary = lambda x: x
            .groupby(['report_dt', 'shop_id', 'pos_name'])
            ['salary']
            .transform('mean'))
    .iloc[:, [0, 1, 2, 3, 4, 5, -2, -1]]
    .sample(5, random_state=10)
)

,report_dt,i_pernr,fio,sex,shop_id,pos_name,salary,mean_group_salary
4663,2018-03-31,311,Русакова Вероника Артёмовна,женский,6,Продавец-консультант,25100.0,24700.000000
8326,2019-02-28,510,Зубкова Мария Владиславовна,женский,2,Продавец-консультант,44200.0,44933.333333
15824,2020-08-31,1067,Судакова Екатерна Николаевна,женский,6,Кассир,18800.0,19425.000000
3116,2017-09-30,287,Сомова Анна Николаевна,женский,1,Продавец-консультант,20700.0,25914.285714
10325,2019-07-31,432,Горелов Алексей Маратович,мужской,6,Кассир,24800.0,23237.500000


Вычтем из зарплаты среднюю зарпалату по группе и добавим в качестве новой колонки

In [31]:
(empl
    .assign(mean_group_salary_diff = lambda x: x
            .groupby(['report_dt', 'shop_id', 'pos_name'])
            ['salary']
            .transform(lambda x: x - x.mean()))
    .iloc[:, [0, 1, 2, 3, 4, 5, -2, -1]]
    .sample(5, random_state=10)
)

,report_dt,i_pernr,fio,sex,shop_id,pos_name,salary,mean_group_salary_diff
4663,2018-03-31,311,Русакова Вероника Артёмовна,женский,6,Продавец-консультант,25100.0,400.000000
8326,2019-02-28,510,Зубкова Мария Владиславовна,женский,2,Продавец-консультант,44200.0,-733.333333
15824,2020-08-31,1067,Судакова Екатерна Николаевна,женский,6,Кассир,18800.0,-625.000000
3116,2017-09-30,287,Сомова Анна Николаевна,женский,1,Продавец-консультант,20700.0,-5214.285714
10325,2019-07-31,432,Горелов Алексей Маратович,мужской,6,Кассир,24800.0,1562.500000


Добавлять несколько колонок можно при помощи метода join

In [32]:
(empl
    .join(empl
              .groupby(['report_dt', 'shop_id', 'pos_name'])
              [['employee_evaluation', 'salary']]
              .transform(lambda x: x - x.mean()),
            rsuffix='_centered')
    .sample(5, random_state=10)
)

,report_dt,i_pernr,fio,sex,shop_id,pos_name,birth_date,education,employee_evaluation,salary,employee_evaluation_centered,salary_centered
4663,2018-03-31,311,Русакова Вероника Артёмовна,женский,6,Продавец-консультант,1970-08-19,среднее общее,6,25100.0,0.2,400.0
8326,2019-02-28,510,Зубкова Мария Владиславовна,женский,2,Продавец-консультант,1975-07-06,среднее профессиональное,6,44200.0,0.333333,-733.333333
15824,2020-08-31,1067,Судакова Екатерна Николаевна,женский,6,Кассир,1965-10-21,основное общее,6,18800.0,0.714286,-625.0
3116,2017-09-30,287,Сомова Анна Николаевна,женский,1,Продавец-консультант,1965-08-27,среднее общее,5,20700.0,-0.714286,-5214.285714
10325,2019-07-31,432,Горелов Алексей Маратович,мужской,6,Кассир,1984-06-16,среднее общее,5,24800.0,-0.875,1562.5


## groupby(...).filter

Отфильтровывает определенные группы на основе условий

Отбираем людей, которые встречаются в таблице 24 раза или более

In [33]:
(empl
    .groupby('i_pernr')
    # внутрь метода передается функция, которая возвращает True или False
    # для каждой группы
    .filter(lambda x: len(x) > 24)
    .head()
)

,report_dt,i_pernr,fio,sex,shop_id,pos_name,birth_date,education,employee_evaluation,salary
7,2015-01-31,6,Леонов Николай Артурович,мужской,1,Продавец-консультант,1966-02-18,среднее профессиональное,<NA>,15400.0
14,2015-02-28,9,Носова Алена Захаровна,женский,1,Кассир,1990-11-16,среднее профессиональное,<NA>,12700.0
18,2015-02-28,6,Леонов Николай Артурович,мужской,1,Продавец-консультант,1966-02-18,среднее профессиональное,6,15400.0
26,2015-03-31,9,Носова Алена Захаровна,женский,1,Кассир,1990-11-16,среднее профессиональное,6,12700.0
31,2015-03-31,6,Леонов Николай Артурович,мужской,1,Продавец-консультант,1966-02-18,среднее профессиональное,6,15400.0


Отбираем людей, которые побывали на нескольких должностях

In [34]:
(empl
    .groupby('i_pernr')
    .filter(lambda x: x['pos_name'].nunique() >= 2)
    .sort_values(by='i_pernr')
    .head()
)

,report_dt,i_pernr,fio,sex,shop_id,pos_name,birth_date,education,employee_evaluation,salary
6,2015-01-31,5,Грачев Даниил Артёмович,мужской,1,Продавец-консультант,1987-02-27,основное общее,<NA>,15400.0
939,2016-07-31,5,Грачев Даниил Артёмович,мужской,1,Кассир,1987-02-27,основное общее,6,17800.0
1051,2016-08-31,5,Грачев Даниил Артёмович,мужской,1,Кассир,1987-02-27,основное общее,6,17800.0
714,2016-05-31,5,Грачев Даниил Артёмович,мужской,1,Кассир,1987-02-27,основное общее,6,17800.0
1158,2016-09-30,5,Грачев Даниил Артёмович,мужской,1,Кассир,1987-02-27,основное общее,6,17800.0


## groupby(...).apply

In [35]:
(empl
    .groupby(['report_dt', 'shop_id'])
    .apply(lambda x: (x['report_dt'] - x['birth_date']).dt.days.mean() / 365)
    .reset_index()
)

,report_dt,shop_id,0
0,2015-01-31,1,36.893151
1,2015-02-28,1,35.226027
2,2015-03-31,1,36.391781
3,2015-04-30,1,36.473973
4,2015-04-30,2,32.809741
...,...,...,...
1445,2022-12-31,19,47.681018
1446,2022-12-31,20,46.960731
1447,2022-12-31,21,42.307721
1448,2022-12-31,22,39.167995


С помощью метода apply можно делать агрегацию с условиями

In [36]:
(empl
    .groupby(['report_dt', 'shop_id'])
    .apply(lambda x: pd.Series({'зп высшее образование': x.query('education == "высшее"')['salary'].mean(),
                                'зп все': x['salary'].mean()}))
    .reset_index()
)

,report_dt,shop_id,зп высшее образование,зп все
0,2015-01-31,1,45700.000000,18330.000000
1,2015-02-28,1,45700.000000,17625.000000
2,2015-03-31,1,45700.000000,17253.846154
3,2015-04-30,1,45700.000000,17515.384615
4,2015-04-30,2,46616.666667,35116.666667
...,...,...,...,...
1445,2022-12-31,19,33566.666667,31078.571429
1446,2022-12-31,20,95700.000000,35900.000000
1447,2022-12-31,21,80033.333333,37031.818182
1448,2022-12-31,22,55700.000000,37659.090909


Использование apply вместе с методами, которые не реализованы внутри groupby

Возьмем строки только за первый календарный год работы магазина

In [37]:
(empl
    .groupby(['shop_id'], as_index=False)
    .apply(lambda x: x.set_index('report_dt').first('Y'))
    .droplevel(0)
    .reset_index()
)

,report_dt,i_pernr,fio,sex,shop_id,pos_name,birth_date,education,employee_evaluation,salary
0,2015-01-31,100000,Кожевников Андрей Максимович,мужской,1,Директор магазина,1987-03-11,высшее,5,45700.0
1,2015-01-31,0,Дубинина Надежда Николаевна,женский,1,Старший кассир,1971-07-07,среднее профессиональное,5,20000.0
2,2015-01-31,1,Вишневскиая Валентина Константиновна,женский,1,Кассир,1974-11-29,среднее общее,5,12600.0
3,2015-01-31,2,Литвинов Александр Максимович,мужской,1,Кассир,1969-11-01,среднее общее,3,12600.0
4,2015-01-31,3,Черняева Валерия Глебовна,женский,1,Продавец-консультант,1988-07-17,среднее общее,<NA>,15400.0
...,...,...,...,...,...,...,...,...,...,...
3150,2020-12-31,1263,Баранова Елена Дмитриевна,женский,23,Продавец-консультант,2001-07-21,основное общее,4,23400.0
3151,2020-12-31,1264,Бессонова Вера Артуровна,женский,23,Продавец-консультант,1977-01-09,среднее общее,5,23400.0
3152,2020-12-31,1265,Чеснокова Екатерна Олеговна,женский,23,Товаровед,1974-09-21,среднее общее,4,26000.0
3153,2020-12-31,1266,Игнатьев Фёдор Константинович,мужской,23,Товаровед,1965-03-17,среднее профессиональное,5,26000.0


### Список методов, которые можно использовать после groupby
#### Они все вычисляются в перделах одной группы

#### Статистики (можно применять внутри agg):
1. __mean__ - среднее
2. __median__ - медиана
3. __min__ - минимум
4. __max__ - максимум
5. __std__ - стандартное отклонение
6. __var__ - дисперсия
7. __nunique__ - число уникальных значений
8. __count__ - число заполненных строк
9. __size__ - число строк
10. __sum__ - сумма
11. __prod__ - произведние
12. __skew__ - ассиметрия
13. __quantile__ - квантиль (внутри agg будет считать медиану)
14. __first__ - первое заполненное значение в колонке
15. __last__ - последнее заполненное значение в колонке
16. __nth__ - возвращает значение под номером n (нельзя использовать внутри agg)
17. __mad__ - среднее отклонение от среднего (df - df.mean()).abs().mean()
18. __sem__ - стандартное отклонение среднего
19. __idxmin__ - индекс минимума
20. __idxmax__ - индекс максимума
21. __ohls__ - создает колонки с первым, последним, минимальным и максимальным значением внути группы
22. __corr__ - коэффициент корреляции
23. __cov__ - ковариация
24. __all__ - проверяет, что все значения равны True
25. __any__ - проверяет, что хотя бы одно значение равно True
26. __value_counts__ - число уникальных значений в каждом столбце
27. __describe__ - возвращает число заполненный строк, среднее, стандартное отклонение, минимум, 25, 50, 75 квантиль и максимум
28. __dtypes__ - тип данных (только для датафремов) и после него скобки ставить не надо

#### Преобразование данных:
1. __cummin__ - накопленный минимум
2. __cummax__ - накопеленный максимум
3. __cumsum__ - накопленная сумма
4. __cumprod__ - накопленное произведние
5. __cumcount__ - наколенное число заполненный строк
6. __rank__ - ранг
7. __shift__ - сдвиг столбца на несколько строк
8. __diff__ - разность между текущей и предыдущей строчкой
9. __pct_change__ - процентное (в долях) изменение между текущей и предыдущей строчкой
10. __bfill__ или __backfill__ - заполнение пропущенного значения предыдущим заполненным (тем, что ниже по стобцу)
11. __ffill__ или __pad__ - заполнение пропущенного значения следующим заполненным (тем, что выше по столбцу)
12. __fillna__ - заполнение пропущенного значения произвольным значением

#### Окна:
1. __resample__ - окно, сформированное на основе даты и времени
2. __rolling__ - скользящее окно
3. __expanding__ - расширяющееся окно
4. __ewm__ - окно с экспоненциально взвешенными операциями

#### Выбор данных:
1. __head__ - возвращает первые n строк (или долю строк) из каждой группы
2. __tail__ - возвращает последние n строк (или долю строк) из каждой группы
3. __sample__ - возвращает случайные n строк (или долю строк) из каждой группы

#### Методы для применения функций:
1. __pipe__ - применение функции к датафрейму, которая возвращает датафрейм
2. __apply__ - применение функции к датафрейму
3. __filter__ - фильтация данных
5. __transform__ - трансформация данных
4. __agg__ или __aggregate__ - расчет статистик

#### Построение графиков:
1. __plot__ - график
2. __boxplot__ - ящик с усами
3. __hist__ - гистограмма

#### Методы группы:
1. __ngroups__ - возвращает число групп
2. __get_group__ - возвращает группу по ее индексу
3. __groups__ - возвращает словарь с индексами групппы

#### Методы только для Series
1. __is_monotonic_increasing__ - возвращает True если последовательность внутри группы возрастает, иначе False
2. __is_monotonic_decreasing__ - возвращает True если последовательность внутри группы убывает, иначе False

# Задачи

#### Описание таблиц лежит [здесь](https://github.com/victorlymarev/pandas/tree/main/tables)

Здесь задач довольно много. Они условно разделены на легкие и  на те, что посложнее. Вам необязательно решать их все. Сначала просмотрите легкие задачи. Выберите из них самые сложные и решите их.

Затем просмотрите задачи посложнее и попробуйте прорешать их. Самые сложные задачи находятся ближе к концу.

Некоторые таблицы занимают много памяти, поэтому каждые 5-10 заданий лучше перезапускайте ноутбук.

Если вы будете работать с этим ноутбуком в google colab и у вас что-то не будет работать, то раскоментируйте ячейку ниже (для этого выделите содержимое ячейки и нажмите на клавиши ctrl / (если смотреть по английской раскладке или ctrl . для русской), либо просто руками уберите # и пробел после нее в каждой строчке) и запустите ее.

In [ ]:
# import os
# os._exit(00)
# !pip install pyarrow pandas==1.5.3

## Легкие 

### Задача 1

Посчитайте сколько людей уволилось в каждом месяце каждого года

In [ ]:
import os
import pandas as pd

path_events = '../tables/events.parquet' if os.path.exists('../tables/events.parquet') else 'https://drive.google.com/uc?id=1Phn06U4-BI0XqtYAgtlM6IuoXRIAGQlO'

events = pd.read_parquet(path_events)
events.head()

In [ ]:
# напишите свой код здесь

### Задача 2

Посчитать долю женщин среди менеджмента и среди обычных сотрудников (по полю mgmt) по состоянию на декабрь каждого года

In [ ]:
import os
import pandas as pd

path_empl = '../tables/employees.parquet' if os.path.exists('../tables/employees.parquet') else 'https://drive.google.com/uc?id=1AARD5-eVlCxoApt5CYZebrC3Cqw42lvj'

empl = pd.read_parquet(path_empl)
empl.head()

In [ ]:
# напишите свой код здесь

### Задача 3

Посчитайте долю пропущенных значений за каждый месяц в каждой колонке

In [ ]:
import os
import pandas as pd

path_empl = '../tables/employees.parquet' if os.path.exists('../tables/employees.parquet') else 'https://drive.google.com/uc?id=1AARD5-eVlCxoApt5CYZebrC3Cqw42lvj'

empl = pd.read_parquet(path_empl)
empl.head()

In [ ]:
# напишите свой код здесь

### Задача 4

Посчитать какой процент покупателей в каждом магазине не пользуется услугами продавца-консультанта

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_2022 на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_2022 = '../tables/sales_2022.parquet' if os.path.exists('../tables/sales_2022.parquet') else 'https://drive.google.com/uc?id=17e7FwXVdsWc2aziK9s5KidIvPcfKt9F5'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_2022)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 5

Для каждого чека, посчитайте среднюю стоимость одного товара в чеке без учета количества купленных товаров

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_sample_check на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_sample_check = '../tables/sales_sample_check.parquet' if os.path.exists('../tables/sales_sample_check.parquet') else 'https://drive.google.com/uc?id=1oYT518oqGnEF51PSFHfSHYNP-690ktFL'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample_check)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 6

Постройте ящичковую диаграмму (boxplot) для суммы чека

О том, как строить boxplot, рассказывается в 13 ноутбуке, где мы смотрели о статистиках

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_sample_check на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_sample_check = '../tables/sales_sample_check.parquet' if os.path.exists('../tables/sales_sample_check.parquet') else 'https://drive.google.com/uc?id=1oYT518oqGnEF51PSFHfSHYNP-690ktFL'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample_check)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 7

Для каждого чека, посчитайте среднюю стоимость одного товара в чеке с учетом количества купленных товаров

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_sample_check на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_sample_check = '../tables/sales_sample_check.parquet' if os.path.exists('../tables/sales_sample_check.parquet') else 'https://drive.google.com/uc?id=1oYT518oqGnEF51PSFHfSHYNP-690ktFL'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample_check)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 8

Посчитайте средний, медианный чек за каждый день недели, а так же 10 и 90 квантили

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_2022 на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_2022 = '../tables/sales_2022.parquet' if os.path.exists('../tables/sales_2022.parquet') else 'https://drive.google.com/uc?id=17e7FwXVdsWc2aziK9s5KidIvPcfKt9F5'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_2022)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 9

За каждый месяц выберите 50 чеков с наибольшей суммой покупки


<details>

<summary>Подсказка</summary>
 
Здесь необходимо делать 2 группировки подряд. Сначала посчитать сумму каждого чека, а затем сделать группировку по месяцу

</details>

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_2022 на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_2022 = '../tables/sales_2022.parquet' if os.path.exists('../tables/sales_2022.parquet') else 'https://drive.google.com/uc?id=17e7FwXVdsWc2aziK9s5KidIvPcfKt9F5'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_2022)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 10

Вывидите категори товаров (поле product_type), которые представлены только для женщин (например платья)

In [ ]:
import os
import pandas as pd

path_goods_descr = '../tables/goods_description.parquet' if os.path.exists('../tables/goods_description.parquet') else 'https://drive.google.com/uc?id=1YbiD02Rev-X_WWV9nPSG1zZFmEh2JjPh'

goods_descr = pd.read_parquet(path_goods_descr)
goods_descr.head()

In [ ]:
# напишите свой код здесь

### Задача 11

Добавьте к датафрейму колонку, со средним числом товаров в чеке для каждого покупателя

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_2022 на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_2022 = '../tables/sales_2022.parquet' if os.path.exists('../tables/sales_2022.parquet') else 'https://drive.google.com/uc?id=17e7FwXVdsWc2aziK9s5KidIvPcfKt9F5'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_2022)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 12

Найти людей которые работали в разных магазинах

In [ ]:
import os
import pandas as pd

path_empl = '../tables/employees.parquet' if os.path.exists('../tables/employees.parquet') else 'https://drive.google.com/uc?id=1AARD5-eVlCxoApt5CYZebrC3Cqw42lvj'

empl = pd.read_parquet(path_empl)
empl.head()

In [ ]:
# напишите свой код здесь

### Задача 13

Посчитайте насколько рублей зарплата сотрудника отличается от средней зарплаты сотрудников по команде

In [ ]:
import os
import pandas as pd

path_empl = '../tables/employees.parquet' if os.path.exists('../tables/employees.parquet') else 'https://drive.google.com/uc?id=1AARD5-eVlCxoApt5CYZebrC3Cqw42lvj'

empl = pd.read_parquet(path_empl)
empl.head()

In [ ]:
# напишите свой код здесь

### Задача 14

Посчитайте среднюю оценку сотрудников за каждый месяц с разбивкой по уровню образования. Приведите таблицу к виду сводной таблицы. В названии колонок должен лежать уровень образования, а в индексе дата

In [ ]:
import os
import pandas as pd

path_empl = '../tables/employees.parquet' if os.path.exists('../tables/employees.parquet') else 'https://drive.google.com/uc?id=1AARD5-eVlCxoApt5CYZebrC3Cqw42lvj'

empl = pd.read_parquet(path_empl)
empl.head()

In [ ]:
# напишите свой код здесь

### Задача 15

Выведите таблицу по продажам для людей, которые были у нас в магазине 10 и более раз

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_sample на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_sample = '../tables/sales_sample.parquet' if os.path.exists('../tables/sales_sample.parquet') else 'https://drive.google.com/uc?id=1BgxAxPauzC4LObY3N65PgAvnU3f5QRj2'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 16

Из неравенства Чебышева,  следует, что вероятность того, что показатель отклониться от своего среднего более чем на 3 стандартных отклонения не превышает 1/9. Вычислите суммарный чек за каждую покупку и посчитайте, какой процент наблюдений выходит за границу трех страндартных отклонений

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_2022 на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_2022 = '../tables/sales_2022.parquet' if os.path.exists('../tables/sales_2022.parquet') else 'https://drive.google.com/uc?id=17e7FwXVdsWc2aziK9s5KidIvPcfKt9F5'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_2022)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 17

Постройте точечный график (scatter) зависимости числа продаж товара (считайте по полю product_code_2) от его цены

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_2022 на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_2022 = '../tables/sales_2022.parquet' if os.path.exists('../tables/sales_2022.parquet') else 'https://drive.google.com/uc?id=17e7FwXVdsWc2aziK9s5KidIvPcfKt9F5'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_2022)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 18

Постройте график среднего числа посетителей в крупных магазинах за каждый день недели

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_2022 на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_2022 = '../tables/sales_2022.parquet' if os.path.exists('../tables/sales_2022.parquet') else 'https://drive.google.com/uc?id=17e7FwXVdsWc2aziK9s5KidIvPcfKt9F5'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_2022)
sales.head()

In [ ]:
# напишите свой код здесь

In [ ]:
path_shops = '../tables/shops.xlsx' if os.path.exists('../tables/shops.xlsx') else 'https://drive.google.com/uc?id=1gfnmceJa3Mc1X06NftTx9G9QfKfprjEB'

shops = pd.read_excel(path_shops)
shops.head()

In [ ]:
# напишите свой код здесь

### Задача 19

Посчитайте средний месячный товарооборот (сумма продаж за месяц) по средним крупным и маленьким магазинам

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_2022 на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_2022 = '../tables/sales_2022.parquet' if os.path.exists('../tables/sales_2022.parquet') else 'https://drive.google.com/uc?id=17e7FwXVdsWc2aziK9s5KidIvPcfKt9F5'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_2022)
sales.head()

In [ ]:
path_shops = '../tables/shops.xlsx' if os.path.exists('../tables/shops.xlsx') else 'https://drive.google.com/uc?id=1gfnmceJa3Mc1X06NftTx9G9QfKfprjEB'

shops = pd.read_excel(path_shops)
shops.head()

In [ ]:
# напишите свой код здесь

### Задача 20

По каждому клиенту посчитать средний чек, среднее число товаров в чеке, число посещений, максимальный и минимальный чек

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_2022 на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_2022 = '../tables/sales_2022.parquet' if os.path.exists('../tables/sales_2022.parquet') else 'https://drive.google.com/uc?id=17e7FwXVdsWc2aziK9s5KidIvPcfKt9F5'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_2022)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 21

Найдите среднее время покупки в выходной и в рабочий день

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_2022 на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_2022 = '../tables/sales_2022.parquet' if os.path.exists('../tables/sales_2022.parquet') else 'https://drive.google.com/uc?id=17e7FwXVdsWc2aziK9s5KidIvPcfKt9F5'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_2022)
sales.head()

In [ ]:
path_w_ends = '../tables/выходные.csv' if os.path.exists('../tables/выходные.csv') else 'https://drive.google.com/uc?id=1hAsoXSPTFihFmBuF-E0uSj9pX4gpojwC'

w_ends = pd.read_csv(path_w_ends)
w_ends.head()

In [ ]:
# напишите свой код здесь

### Задача 22

Нарисуйте графики среднего числа посетителей за каждый час для выходных и для рабочих дней

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_2022 на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_2022 = '../tables/sales_2022.parquet' if os.path.exists('../tables/sales_2022.parquet') else 'https://drive.google.com/uc?id=17e7FwXVdsWc2aziK9s5KidIvPcfKt9F5'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_2022)
sales.head()

In [ ]:
path_w_ends = '../tables/выходные.csv' if os.path.exists('../tables/выходные.csv') else 'https://drive.google.com/uc?id=1hAsoXSPTFihFmBuF-E0uSj9pX4gpojwC'

w_ends = pd.read_csv(path_w_ends)
w_ends.head()

In [ ]:
# напишите свой код здесь

### Задача 23

Для каждого сотрудника, который работал в компании 31 декабря 2021 года посчитайте, сколько раз он устраивался на работу по состоянию на эту дату

In [ ]:
import os
import pandas as pd

path_events = '../tables/events.parquet' if os.path.exists('../tables/events.parquet') else 'https://drive.google.com/uc?id=1Phn06U4-BI0XqtYAgtlM6IuoXRIAGQlO'

events = pd.read_parquet(path_events)
events.head()

In [ ]:
path_empl = '../tables/employees.parquet' if os.path.exists('../tables/employees.parquet') else 'https://drive.google.com/uc?id=1AARD5-eVlCxoApt5CYZebrC3Cqw42lvj'

empl = pd.read_parquet(path_empl)
empl.head()

In [ ]:
# напишите свой код здесь

### Задача 24

Для каждого дня и каждого магазина, посчитайте сколько уникальных товаров было куплено

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_2022 на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_2022 = '../tables/sales_2022.parquet' if os.path.exists('../tables/sales_2022.parquet') else 'https://drive.google.com/uc?id=17e7FwXVdsWc2aziK9s5KidIvPcfKt9F5'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_2022)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 25

Найдите среднее количество размеров у одного товара (группируйте по полю product_code_2)

In [ ]:
import os
import pandas as pd

path_goods_descr = '../tables/goods_description.parquet' if os.path.exists('../tables/goods_description.parquet') else 'https://drive.google.com/uc?id=1YbiD02Rev-X_WWV9nPSG1zZFmEh2JjPh'

goods_descr = pd.read_parquet(path_goods_descr)
goods_descr.head()

In [ ]:
# напишите свой код здесь

### Задача 26

Найти людей, которые хоть раз делали 3 и более покупок за день (у покупателя должно быть 3 и более чека за день)

In [ ]:
import os
import pandas as pd

path_sales_sample = '../tables/sales_sample.parquet' if os.path.exists('../tables/sales_sample.parquet') else 'https://drive.google.com/uc?id=1BgxAxPauzC4LObY3N65PgAvnU3f5QRj2'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample)
sales.head()

In [ ]:
# напишите свой код здесь

## Задачи посложнее

### Задача 27

Для каждого человека посчитать с какой периодичностью он приходит в магазин.

Для этого для каждого customer_id, который был у нас 2 и более раз (для него было 2 чека и более) вычтите из даты последнего посещения дату первого посещения, и разделите это значение на число посещений. 
Полученную колонку назовите mean_period. Округлите значение до числа дней, и сохраните как целое число

Индекс в таблице должен быть стандартный

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_sample на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_sample = '../tables/sales_sample.parquet' if os.path.exists('../tables/sales_sample.parquet') else 'https://drive.google.com/uc?id=1BgxAxPauzC4LObY3N65PgAvnU3f5QRj2'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 28

Посчитайте для каждого месяца и каждого магазина какой процент клиентов возвращается в магазин в течение следующих 6 месяцев

In [ ]:
import os
import pandas as pd

path_sales_sample = '../tables/sales_sample.parquet' if os.path.exists('../tables/sales_sample.parquet') else 'https://drive.google.com/uc?id=1BgxAxPauzC4LObY3N65PgAvnU3f5QRj2'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 29

На конец каждого месяца посчитайте, сколько уникальных клиентов в пришло в магазины а так же, какой процент клиентов приходил 2 и более раза за месяц

In [ ]:
import os
import pandas as pd

path_sales_sample = '../tables/sales_sample.parquet' if os.path.exists('../tables/sales_sample.parquet') else 'https://drive.google.com/uc?id=1BgxAxPauzC4LObY3N65PgAvnU3f5QRj2'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 30

Найти людей, которые 3 и более раз делали 2 и более покупок за день (у покупателя должно быть 2 и более чека за день 3 раза)

In [ ]:
import os
import pandas as pd

path_sales_sample = '../tables/sales_sample.parquet' if os.path.exists('../tables/sales_sample.parquet') else 'https://drive.google.com/uc?id=1BgxAxPauzC4LObY3N65PgAvnU3f5QRj2'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 31

Для сотрудников, которые увольнялись и приходили обратно посчитать среднее число дней, через которое они возвращались. (если таких увольнений и приходов было несколько)

In [ ]:
import os
import pandas as pd

path_events = '../tables/events.parquet' if os.path.exists('../tables/events.parquet') else 'https://drive.google.com/uc?id=1Phn06U4-BI0XqtYAgtlM6IuoXRIAGQlO'

events = pd.read_parquet(path_events)
events.head()

In [ ]:
# напишите свой код здесь

### Задача 32

Посчитайте, какое количество товаров пробил каждый продавец за каждый месяц. Затем посчитайте долю товаров, которую пробил каждый продавец среди общего числа пробитых товаров по магазину в месяц.

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_2022 на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_2022 = '../tables/sales_2022.parquet' if os.path.exists('../tables/sales_2022.parquet') else 'https://drive.google.com/uc?id=17e7FwXVdsWc2aziK9s5KidIvPcfKt9F5'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_2022)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 33

Найдите товары, у которых разные цвета стоят по-разному

In [ ]:
import os
import pandas as pd

path_goods_descr = '../tables/goods_description.parquet' if os.path.exists('../tables/goods_description.parquet') else 'https://drive.google.com/uc?id=1YbiD02Rev-X_WWV9nPSG1zZFmEh2JjPh'

goods_descr = pd.read_parquet(path_goods_descr)
goods_descr.head()

In [ ]:
path_prices = '../tables/prices.parquet' if os.path.exists('../tables/prices.parquet') else 'https://drive.google.com/uc?id=1_KfYyYCsib8woanbP3VouLUpYp103dV5'

prices = pd.read_parquet(path_prices)
prices.head()

In [ ]:
# напишите свой код здесь

### Задача 34

Для каждого руководителя посчитайте число подчиненных за каждый месяц (по колонке closest_boss_id), затем усредните результат для каждого руководителя

In [ ]:
import os
import pandas as pd

path_empl = '../tables/employees.parquet' if os.path.exists('../tables/employees.parquet') else 'https://drive.google.com/uc?id=1AARD5-eVlCxoApt5CYZebrC3Cqw42lvj'

empl = pd.read_parquet(path_empl)
empl.head()

In [ ]:
# напишите свой код здесь

### Задача 35

Выбирите сотрудников, которые работают в 2 раза дольше чем в среднем по магазину

In [ ]:
import os
import pandas as pd

path_empl = '../tables/employees.parquet' if os.path.exists('../tables/employees.parquet') else 'https://drive.google.com/uc?id=1AARD5-eVlCxoApt5CYZebrC3Cqw42lvj'

empl = pd.read_parquet(path_empl)
empl.head()

In [ ]:
path_events = '../tables/events.parquet' if os.path.exists('../tables/events.parquet') else 'https://drive.google.com/uc?id=1Phn06U4-BI0XqtYAgtlM6IuoXRIAGQlO'

events = pd.read_parquet(path_events)
events.head()

In [ ]:
# напишите свой код здесь

### Задача 36

Найти людей, которые приобретали один и тот же товар (product_id), но в разных чеках

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_2022 на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_2022 = '../tables/sales_2022.parquet' if os.path.exists('../tables/sales_2022.parquet') else 'https://drive.google.com/uc?id=17e7FwXVdsWc2aziK9s5KidIvPcfKt9F5'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_2022)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 37

Выведете имена 5 самых результативных продовцов-консультантов, которые помогли заработать магазину больше всего денег

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_2022 на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_2022 = '../tables/sales_2022.parquet' if os.path.exists('../tables/sales_2022.parquet') else 'https://drive.google.com/uc?id=17e7FwXVdsWc2aziK9s5KidIvPcfKt9F5'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_2022)
sales.head()

In [ ]:
path_empl = '../tables/employees.parquet' if os.path.exists('../tables/employees.parquet') else 'https://drive.google.com/uc?id=1AARD5-eVlCxoApt5CYZebrC3Cqw42lvj'

empl = pd.read_parquet(path_empl)
empl.head()

In [ ]:
# напишите свой код здесь

### Задача 38

По каждой категории (product_type) и каждому бренду найти насколько средняя цена на мужские товары отличается от средней цены на женские товары. 

Для этого сделайте группировку по product_code_2, и выкините дубликаты, чтобы разные размеры учитывались как один товар

In [ ]:
import os
import pandas as pd

path_goods_descr = '../tables/goods_description.parquet' if os.path.exists('../tables/goods_description.parquet') else 'https://drive.google.com/uc?id=1YbiD02Rev-X_WWV9nPSG1zZFmEh2JjPh'

goods_descr = pd.read_parquet(path_goods_descr)
goods_descr.head()

In [ ]:
path_prices = '../tables/prices.parquet' if os.path.exists('../tables/prices.parquet') else 'https://drive.google.com/uc?id=1_KfYyYCsib8woanbP3VouLUpYp103dV5'

prices = pd.read_parquet(path_prices)
prices.head()

In [ ]:
# напишите свой код здесь

### Задача 39

Товары с каким цветом стоят дороже всего (не учитывайте размер товара)?

In [ ]:
import os
import pandas as pd

path_goods_descr = '../tables/goods_description.parquet' if os.path.exists('../tables/goods_description.parquet') else 'https://drive.google.com/uc?id=1YbiD02Rev-X_WWV9nPSG1zZFmEh2JjPh'

goods_descr = pd.read_parquet(path_goods_descr)
goods_descr.head()

In [ ]:
# напишите свой код здесь

### Задача 40

Найдите сотрудников которым ни разу не повышали зарплату

In [ ]:
import os
import pandas as pd

path_empl = '../tables/employees.parquet' if os.path.exists('../tables/employees.parquet') else 'https://drive.google.com/uc?id=1AARD5-eVlCxoApt5CYZebrC3Cqw42lvj'

empl = pd.read_parquet(path_empl)
empl.head()

In [ ]:
path_events = '../tables/events.parquet' if os.path.exists('../tables/events.parquet') else 'https://drive.google.com/uc?id=1Phn06U4-BI0XqtYAgtlM6IuoXRIAGQlO'

events = pd.read_parquet(path_events)
events.head()

In [ ]:
# напишите свой код здесь

### Задача 41

Существует формула, согласно которой общая дисперсия (посчитанная по всей выборке) равна среднему внутригруповых дисперсий (дисперсия посчитанных по каждой группе) сложенному с межгруповой дисперсией (дисперсией внутригрупповых средних).



E - это среднее

D - это дисперсия

Иными словами дисперсия равна среднему условных дисперсий (условная дисперсия это дисперсия внутри группы) плюс дисперсии средних значений по каждой группе

Проверьте этот факт на примере стоимости товаров разных категорий, на 31 декабря 2022 года.

Объясните почему дисперсии не сходятся, и что нужно сделать, чтобы они сходились?

In [ ]:
import os
import pandas as pd

path_goods_descr = '../tables/goods_description.parquet' if os.path.exists('../tables/goods_description.parquet') else 'https://drive.google.com/uc?id=1YbiD02Rev-X_WWV9nPSG1zZFmEh2JjPh'

goods_descr = pd.read_parquet(path_goods_descr)
goods_descr.head()

In [ ]:
path_prices = '../tables/prices.parquet' if os.path.exists('../tables/prices.parquet') else 'https://drive.google.com/uc?id=1_KfYyYCsib8woanbP3VouLUpYp103dV5'

prices = pd.read_parquet(path_prices)
prices.head()

In [ ]:
# напишите свой код здесь

### Задача 42

Посчитайте скидку на каждый товар в чеке. Округлите значение скидки до десятков процентов. Затем для каждого размера скидки посчитайте среднюю долю таких товаров в чеке за каждый месяц.

Сравните результат с таблицей prices. В ней посчитайте срденюю долю товаров за каждый месяц, продаваемых с каждым размером скидки, кругленным до десятков процентов

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_sample на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_sample = '../tables/sales_sample.parquet' if os.path.exists('../tables/sales_sample.parquet') else 'https://drive.google.com/uc?id=1BgxAxPauzC4LObY3N65PgAvnU3f5QRj2'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample)
sales.head()

In [ ]:
path_prices = '../tables/prices.parquet' if os.path.exists('../tables/prices.parquet') else 'https://drive.google.com/uc?id=1_KfYyYCsib8woanbP3VouLUpYp103dV5'

prices = pd.read_parquet(path_prices)
prices.head()

In [ ]:
# напишите свой код здесь

### Задача 43

Найти чеки в которых люди пукупали один и тот же товар, но разных размеров, посчитать процент таких чеков

Колонки product_code_ содержат артикулы товаров

В колонке product_code_3 - каждый размер имеет уникальный артикул

В колонке product_code_2 - каждый цвет имеет уникальный артирул, но при этом у товаров с разным размером он одинаковый

В колонке product_code_2 - артикул товара. Одинаковые товары разных цветов и размеров имеют одинаковый артикул

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_sample на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_sample = '../tables/sales_sample.parquet' if os.path.exists('../tables/sales_sample.parquet') else 'https://drive.google.com/uc?id=1BgxAxPauzC4LObY3N65PgAvnU3f5QRj2'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample)
sales.head()

In [ ]:
# напишите свой код здесь

### Задача 44

Найдите долю покупателей и товары, которые покупали один и тот же товар, но в разных цветах. Необходимо вывести id покупателя product_code_3 (соотвественно у каждого покупателя будет несколько строк)

Колонки product_code_ содержат артикулы товаров

В колонке product_code_3 - каждый размер имеет уникальный артикул

В колонке product_code_2 - каждый цвет имеет уникальный артирул, но при этом у товаров с разным размером он одинаковый

В колонке product_code_2 - артикул товара. Одинаковые товары разных цветов и размеров имеют одинаковый артикул

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_sample на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_sample = '../tables/sales_sample.parquet' if os.path.exists('../tables/sales_sample.parquet') else 'https://drive.google.com/uc?id=1BgxAxPauzC4LObY3N65PgAvnU3f5QRj2'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample)
sales.head()

In [ ]:
path_goods_descr = '../tables/goods_description.parquet' if os.path.exists('../tables/goods_description.parquet') else 'https://drive.google.com/uc?id=1YbiD02Rev-X_WWV9nPSG1zZFmEh2JjPh'

goods_descr = pd.read_parquet(path_goods_descr)
goods_descr.head()

In [ ]:
# напишите свой код здесь

### Задача 45

Для каждого покупателя посчитайте долю мужских и женских вещей в купленной одежде. Если человек купил более 70% мужских товаров, то отметьте его как мужчину, если более 70% его вещей женские, то как женщину. 

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_2022 на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_2022 = '../tables/sales_2022.parquet' if os.path.exists('../tables/sales_2022.parquet') else 'https://drive.google.com/uc?id=17e7FwXVdsWc2aziK9s5KidIvPcfKt9F5'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_2022)
sales.head()

In [ ]:
path_goods_descr = '../tables/goods_description.parquet' if os.path.exists('../tables/goods_description.parquet') else 'https://drive.google.com/uc?id=1YbiD02Rev-X_WWV9nPSG1zZFmEh2JjPh'

goods_descr = pd.read_parquet(path_goods_descr)
goods_descr.head()

In [ ]:
# напишите свой код здесь

### Задача 46

Возьмите каждую категорию и закодируйте день года, в который товар был продан, синусом и косинусом так, чтобы 15 января косинус равнялся единице, а 15 июля минус единице. Через синус закодируйте день покупки так, чтобы 15 января и 15 июля он равнялся нулю, а весной и осенью он принимал значения 1 и -1.

Тогда у летних товаров средний косинус дня продажи будет около около -1, а у зимних около 1, у весенних и осенних косинус будет близок к нулю. Синус будет говорить о том, что тавар осенний или весенний. 

Вычислите средний синус и средний косинус для для продажи каждой категории (product_type).

Можете попробовать по-другому закодировать даты, чтобы сезонность товаров была более интрепретируемой.

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_sample на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_sample = '../tables/sales_sample.parquet' if os.path.exists('../tables/sales_sample.parquet') else 'https://drive.google.com/uc?id=1BgxAxPauzC4LObY3N65PgAvnU3f5QRj2'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample)
sales.head()

In [ ]:
path_goods_descr = '../tables/goods_description.parquet' if os.path.exists('../tables/goods_description.parquet') else 'https://drive.google.com/uc?id=1YbiD02Rev-X_WWV9nPSG1zZFmEh2JjPh'

goods_descr = pd.read_parquet(path_goods_descr)
goods_descr.head()

In [ ]:
# напишите свой код здесь

### Задача 47

Посчитайте инфляцию за период с 31 декабря 2015 года по 31 декабря 2022 года. Для этого сначала для каждого товара и каждого месяца посчитайте изменение цены за каждый месяц (при условии что товар продавался в двух месяцах). Затем найдите инфляцию для каждого месяца. Чтобы посчитать инфляцию за все периоды перемножьте полученные значения

In [ ]:
import os
import pandas as pd

path_prices = '../tables/prices.parquet' if os.path.exists('../tables/prices.parquet') else 'https://drive.google.com/uc?id=1_KfYyYCsib8woanbP3VouLUpYp103dV5'

prices = pd.read_parquet(path_prices)
prices.head()

In [ ]:
# напишите свой код здесь

### Задача 48

Найдите категории товаров в которых процент мужских вещей примерно равен проценту женских. 

Вывидите название категорий, процент мужских товаров в них и процент женских товаров. Таблицу отсортируйте так чтобы категории, внутри которых доля мужских и женских товаров примерно равна находились выше всего

In [ ]:
import os
import pandas as pd

path_goods_descr = '../tables/goods_description.parquet' if os.path.exists('../tables/goods_description.parquet') else 'https://drive.google.com/uc?id=1YbiD02Rev-X_WWV9nPSG1zZFmEh2JjPh'

goods_descr = pd.read_parquet(path_goods_descr)
goods_descr.head()

In [ ]:
# напишите свой код здесь

### Задача 49

Посчитайте суммарную зарплату сотрудников каждого магазина за каждый месяц. Зарплата для нанятых и уволившихся внутри месяца сотрудников расчитывается, как зарплата умноженная на число дней в месяце, когда сотрудники работали и деленная на число дней в месяце. Точно так же считайте зарплату для сотрудников, которым ее повышали.

In [ ]:
import os
import pandas as pd

path_events = '../tables/events.parquet' if os.path.exists('../tables/events.parquet') else 'https://drive.google.com/uc?id=1Phn06U4-BI0XqtYAgtlM6IuoXRIAGQlO'

events = pd.read_parquet(path_events)
events.head()

In [ ]:
path_empl = '../tables/employees.parquet' if os.path.exists('../tables/employees.parquet') else 'https://drive.google.com/uc?id=1AARD5-eVlCxoApt5CYZebrC3Cqw42lvj'

empl = pd.read_parquet(path_empl)
empl.head()

In [ ]:
# напишите свой код здесь

### Задача 50

Узнать среднее время поиска сотрудника на должность. Есть ставки - колонка pos_id. Когда сотрудник увольняется, ставка освобождается до тех пор пока кто-то не устроится. Необходимо найти среднее время, когда ставка пустует.

P.S. Сотрудник у работает как минимум месяц на каждой должности

In [ ]:
import os
import pandas as pd

path_events = '../tables/events.parquet' if os.path.exists('../tables/events.parquet') else 'https://drive.google.com/uc?id=1Phn06U4-BI0XqtYAgtlM6IuoXRIAGQlO'

events = pd.read_parquet(path_events)
events.head()

In [ ]:
path_empl = '../tables/employees.parquet' if os.path.exists('../tables/employees.parquet') else 'https://drive.google.com/uc?id=1AARD5-eVlCxoApt5CYZebrC3Cqw42lvj'

empl = pd.read_parquet(path_empl)
empl.head()

In [ ]:
# напишите свой код здесь

### Задача 51

Насколько в среднем увеличиваются продажи сотрудников после повышения зп. Посчитать средннее отношение суммы и количества проданных товаров в среднем за день за 61 день до повышения зарплаты и через 61 день после повышения зарплаты

In [ ]:
import os
import pandas as pd

path_events = '../tables/events.parquet' if os.path.exists('../tables/events.parquet') else 'https://drive.google.com/uc?id=1Phn06U4-BI0XqtYAgtlM6IuoXRIAGQlO'

events = pd.read_parquet(path_events)
events.head()

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_sample на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_sample = '../tables/sales_sample.parquet' if os.path.exists('../tables/sales_sample.parquet') else 'https://drive.google.com/uc?id=1BgxAxPauzC4LObY3N65PgAvnU3f5QRj2'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample)
sales.head()

In [ ]:
# напишите свой код здесь

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Александр\AppData\Local\Temp\ipykernel_5968\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
